# Pangeo Storage Benchmarks

This is more in depth documentation and discussion of the Pangeo Storage Benchmark tests and some information that will serve as discussion around development of tests and also precursor material for future papers or publications that may result. 

### Scope of Benchmarks
Benchmarks are meant to systematically determine performance characteristics of storage backend/API combinations for processing geoscience data on clustered cloud computing resources based on Xarray/Dask within the Python scientific computing framework. Focus will be primarily on read performance although write tests will be written as well. 

### Use Cases
Here is an overview of the use cases covered by the benchmarks. 

|API/User Interface  | Environment       | format       | Storage Layer     |  
|:------------------ |:----------------- |:-------------|:------------------|
| Dask/Xarray        | GCP               | Zarr         | GCS via gcsfs     |
|                    |                   |              | FUSE              |
|                    |                   | NetCDF4      | FUSE              |
|                    |                   | H5NetCDF     | HSDS/GCS          |
|                    | AWS               | Zarr         | S3 via            |
|                    |                   | NetCDF4      |                   |
|                    |                   | H5NetCDF     | HSDS/S3           |
|                    | GCP               | TileDB       | TBD               |
| Numpy              | GCP               | Zarr         | GCS via GCSFS     |
|                    |                   |              | FUSE              |
|                    |                   | NetCDF4      | FUSE              |
|                    |                   | H5NetCDF     | HSDS/GCS          |
|                    | AWS               | Zarr         | S3 via            |
|                    |                   | NetCDF4      |                   |
|                    |                   | H5NetCDF     | HSDS/S3           |